### we need to map this solution
```
occurs(object(robot, 1), action(move,(0,-1)), 1).
occurs(object(robot, 1), action(move,(0,-1)), 2).  
```
### to this solution

```
occurs(object(robot, 1), action(move,(0,-1)), 1). 
occurs(object(robot, 1), action(move,(0,-1)), 2). 
occurs(object(robot, 1), action(move,(0,-1)), 3).
occurs(object(robot, 2), action(move,(0,-1)), 1).
occurs(object(robot, 2), action(move,(0,-1)), 2).
occurs(object(robot, 2), action(move,(0,-1)), 3).
```

In [139]:
%%file abstract_solution.lp
occurs(object(robot, 1), action(move,(0,-1)), 1).
occurs(object(robot, 1), action(move,(0,-1)), 2).  
occurs(object(robot, 1), action(move,(1,0)), 3).
occurs(object(robot, 1), action(move,(1,0)), 4). 
occurs(object(robot, 1), action(move,(1,0)), 5). 

occurs(object(robot, 2), action(move,(0,-1)), 1).
occurs(object(robot, 2), action(move,(0,-1)), 2).


Overwriting abstract_solution.lp


In [152]:
%%file deabstracter.lp

% find max timestep
t(T) :- occurs(object(robot, ID), action(move,(X,Y)), T).
max_t(T) :- t(T), #max{TT, 1:t(TT)}=T.

% assume goal is reachable by changing direction max 1 time
% mark the first and second direction for each robot
first((X, Y), ID) :- occurs(object(robot, ID), action(move,(X,Y)), 1).
second((X2, Y2), ID) :- occurs(object(robot, ID), action(move,(X2,Y2)), T), max_t(T), first((X,Y), ID2), ID !=ID2.
    
% encode moves into single integer to be able to count them
move(1,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=0, Y=-1.
move(2,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=1, Y=0.
move(3,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=-1, Y=0.
move(4,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=0, Y=1.

% count repetitions of each move for each robot
count((0,-1),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(0,-1)), T) }, N>0.
count((1, 0),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(1,0)), T) }, N>0.
count((-1, 0),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(-1,0)), T) }, N>0.
count((0, 1),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(0,1)), T) }, N>0.

% set the target number of moves in deabstracted solution
goal((X, Y), ID, N*2) :- count((X, Y), ID, N).

% set the range of time-steps for each move for each robot
range((X,Y), ID, (1, M)) :- first((X,Y), ID), goal((X, Y), ID, M).
range((X2,Y2), ID, (M+1, M+M2)) :- second((X2,Y2), ID), goal((X2, Y2), ID, M2), first((X,Y), _), goal((X, Y),_, M).

%finally generate the motion commands based on the range set
new_occurs(object(robot, ID), action(move, (X, Y)), A) :- range((X,Y), ID, (A, _)).
new_occurs(object(robot, ID), action(move, (X, Y)), T+1)  :- new_occurs(object(robot, ID), action(move, (X, Y)), T), range((X,Y), ID, (_, B)), T<B.

Overwriting deabstracter.lp


In [153]:
!clingo deabstracter.lp abstract_solution.lp --out-ifs=\\n 

clingo version 5.6.0
Reading from deabstracter.lp ...
deabstracter.lp:18:57-59: info: global variable in tuple of aggregate element:
  ID

deabstracter.lp:19:57-59: info: global variable in tuple of aggregate element:
  ID

deabstracter.lp:20:58-60: info: global variable in tuple of aggregate element:
  ID

deabstracter.lp:21:57-59: info: global variable in tuple of aggregate element:
  ID

Solving...
Answer: 1
occurs(object(robot,1),action(move,(0,-1)),1)
occurs(object(robot,1),action(move,(0,-1)),2)
occurs(object(robot,1),action(move,(1,0)),3)
occurs(object(robot,1),action(move,(1,0)),4)
occurs(object(robot,1),action(move,(1,0)),5)
occurs(object(robot,2),action(move,(0,-1)),1)
occurs(object(robot,2),action(move,(0,-1)),2)
t(1)
t(2)
t(3)
t(4)
t(5)
max_t(5)
first((0,-1),1)
first((0,-1),2)
second((1,0),1)
move(1,1,1)
move(1,1,2)
move(1,2,1)
move(1,2,2)
move(2,1,3)
move(2,1,4)
move(2,1,5)
count((0,-1),1,2)
count((0,-1),2,2)
count((1,0),1,3)
goal((0,-1),1,4)
goal((0,-1),2,4)
goal((1,0),1,